In [1]:
import multiprocessing as mp
import subprocess
import sys
import os
import itertools
from scipy.io import savemat
from scipy.io import loadmat
import numpy as np

In [2]:
mp.cpu_count()
num_cycles = 6

In [64]:
set_basis_type = ["Z", "X"];
set_error_rate_RO_to_Data = [0.04, 0.03]
set_budget_component = ["None", "CZ", "crosstalk", "SQ", "DD", "idle", "RO_Data", "initial_readout"] ;

args = [(basis, error_rate_RO_to_Data, budget_component) for basis, error_rate_RO_to_Data in zip(set_basis_type, set_error_rate_RO_to_Data)
        for budget_component in set_budget_component]

In [65]:
script = "Budget_sim.py"

def run_Budget_sim(basis_type, error_rate_RO_to_Data, budget_component):
    subprocess.run([sys.executable, script, str(basis_type), str(error_rate_RO_to_Data), str(budget_component)])

with mp.Pool(processes = 50) as pool:
    pool.starmap(run_Budget_sim, args)

## Load the data

In [66]:
labels_z = ['Z0', 'Z1', 'Z2', 'Z5', 'Z6', 'Z7', 'Z8']
num_zcheck = len(labels_z) ;

In [90]:
fname0 = './Numerical_data/' + 'Logical_Z_' + 'budget_' + str("None") + '.mat'
data = loadmat(fname0)
syndrome_history_Z_0 = data["syndrome_history_Z"] 
mean_detect_prob_z_0 = np.mean(syndrome_history_Z_0, axis=0)

set_individual_contribution_Z = {};
set_ave_contr_Z = [];
for budget_component in ["CZ", "crosstalk", "SQ", "DD", "idle", "RO_Data", "initial_readout"] :
    
    fname1 = './Numerical_data/' + 'Logical_Z_' + 'budget_' + str(budget_component) + '.mat'
    data1 = loadmat(fname1)
    # print(data1["retain_data_fraction"][0][0])
    syndrome_history_Z = data1["syndrome_history_Z"] 
    mean_detect_prob_z = np.mean(syndrome_history_Z, axis=0)
    set_individual_contribution_Z[budget_component] = mean_detect_prob_z_0 - mean_detect_prob_z
    set_ave_contr_Z.append( np.mean( mean_detect_prob_z_0 - mean_detect_prob_z ) )

# for key, item in set_individual_contribution_Z.items():
#     print( np.mean(item) )
set_ave_contr_Z = set_ave_contr_Z/sum(set_ave_contr_Z)

In [93]:
set_ave_contr_Z

array([0.24341992, 0.09030479, 0.0626108 , 0.08730291, 0.05589408,
       0.24405136, 0.21641614])

In [60]:
labels_x = ['X0', 'X1', 'X3', 'X4', 'X5', 'X6', 'X7']
num_xcheck = len(labels_x) ;

In [91]:
fnameX = './Numerical_data/' + 'Logical_X_' + 'budget_' + str("None") + '.mat'
data = loadmat(fnameX)
syndrome_history_X_0 = data["syndrome_history_X"] 
mean_detect_prob_x_0 = np.mean(syndrome_history_X_0, axis=0)

set_individual_contribution_X = {};
set_ave_contr_X = [];
for budget_component in ["CZ", "crosstalk", "SQ", "DD", "idle", "RO_Data", "initial_readout"] :
    
    fname1 = './Numerical_data/' + 'Logical_X_' + 'budget_' + str(budget_component) + '.mat'
    data1 = loadmat(fname1)
    syndrome_history_X = data1["syndrome_history_X"] 
    mean_detect_prob_x = np.mean(syndrome_history_X, axis=0)
    set_individual_contribution_X[budget_component] = mean_detect_prob_x_0 - mean_detect_prob_x
    set_ave_contr_X.append( np.mean( mean_detect_prob_x_0 - mean_detect_prob_x ) )

# for key, item in set_individual_contribution_X.items():
#     print( np.mean(item) )
set_ave_contr_X = set_ave_contr_X/sum(set_ave_contr_X)

In [92]:
set_ave_contr_X 

array([0.23517367, 0.1033731 , 0.05742371, 0.15410898, 0.06267703,
       0.1747052 , 0.21253831])